In [ ]:
#Test kernel
6*7

In [ ]:
import os
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import datetime as dt
from datetime import date
from dateutil.relativedelta import relativedelta
import math
#from pprint import pprint 
from IPython.core.display import HTML

import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart

from IPython.display import clear_output

#set directory to import labelLine.py
os.chdir("./")
from labelLine import *

def printHnT(df, x, asHTML=False):
    boolConcat = False if (len(df) < 2*x) else True
    if asHTML:
        if boolConcat:
            display(HTML(pd.concat([df.head(x), df.tail(x)]).to_html()))
        else:
            display(HTML(df.to_html()))
    else:
        if boolConcat:
            print(pd.concat([df.head(x), df.tail(x)]).to_string())
        else:
            print(df.to_string())

# Personalize Here

In [ ]:
#Baby's Name:
babyName = "Wonko"

#Baby's birthday:
babyBDay = pd.Timestamp('2020-01-01 12:00:00')

#Baby's gender (F/M)
babyGender = "M"

#Baby's dataFile
babyFile = "Wonko.xlsx"

#Set working Directory
workDir = "./"
os.chdir(workDir)

#set directory for data
dirData = "" #set to code root

#select category to plot separately
catSel = 5

In [ ]:
#Set up e-mail sending
emailSender = "someAddressThatExists@gmail.com" #smtp set to gmail...
emailReceiver = ['someAddressThatExists@gmail.com', 'anotherAddressThatExists@gmail.com']
password = input("Type your password for sender e-mail address and press enter:")
clear_output()

In [ ]:
emailSubject = babyName+"\s data"
emailBody = "A summary of the baby's growth."

#select plots to attach to mail:
dPics = {
    "WLarge": {"toSend": True},
    "WZoom": {"toSend": True},
    "WZoomSelected": {"toSend": True},
    "HLarge": {"toSend": True},
    "WHLarge": {"toSend": True}
}

#Set data table to send (Weight with closest percentiles)
boolTable = True
daysInTable = 14

# Load Baby's Data 

In [ ]:
excelBaby = pd.ExcelFile(dirData+babyFile)

#Weight
dfW = excelBaby.parse('Weight')

#Weight
dfH = excelBaby.parse('Height')

#TODO: Head Circumference
#dfHC = excelBaby.parse('HeadCirc')

# Load WHO Data

In [ ]:
def fetchWHOData(urlWHO, dirData, fileName):
    print("> Reading WHO data: ", fileName)
    try:
        print(">> Fetching from WHO site")
        return pd.read_csv(urlWHO+fileName,sep="\t")
    except:
        print(">> URL failed. Reading data from file.")
        try:
            return pd.read_csv(dirData+fileName,sep="\t")
        except:
            raise Exception("Error: Missing WHO data:", fileName)

In [ ]:
urlWHO = "https://www.who.int/childgrowth/standards/"

gender = "boys" if (babyGender == "M") else "girls"

#Weight
fileNameWHOWeight = "wfa_"+gender+"_p_exp.txt"
dfWFA = fetchWHOData(urlWHO, dirData, fileNameWHOWeight)

#Height
fileNameWHOHeight02 = "lhfa_"+gender+"_p_exp.txt"
dfHFA = fetchWHOData(urlWHO, dirData, fileNameWHOHeight02)

#lengthe/Height to Weight
fileNameWHOLengtht02 = "wfl_"+gender+"_p_exp.txt"
dfWFH02 = fetchWHOData(urlWHO, dirData, fileNameWHOLengtht02)
fileNameWHOHeight25 = "wfh_"+gender+"_p_exp.txt"
dfWFH25 = fetchWHOData(urlWHO, dirData, fileNameWHOHeight25)

print("> Data Fetching finished.")

#TODO: Head Circumference
#https://www.who.int/childgrowth/standards/hc_for_age/en/

---
# Weight

In [ ]:
dfWFA.rename(columns={"Age": "days"}, inplace=True)
dfWFA["weeks"] = dfWFA['days']/7.
dfWFA["DT"] = dfWFA["days"].apply(lambda x: babyBDay + dt.timedelta(days=x))
dfWFA["Date"] = dfWFA["DT"].apply(lambda x: x.date())
#dfWFA.head()

In [ ]:
dfW["DT"] = pd.to_datetime(dfW["DT"], format='%Y.%m.%d %H:%M')
dfW["Date"] = dfW["DT"].apply(lambda x: x.date())
#dfW["Date"] = pd.to_datetime(dfW["Date"], format='%Y.%m.%d')
dfW["W"] = dfW["W"] / 1000.
dfW["Err"] = dfW["Err"] / 1000.
#dfW["days"] = (dfW['Date'] - bDay).dt.days
dfW["days"] = (86400*((dfW['DT'] - babyBDay).dt.days)+((dfW['DT'] - babyBDay).dt.seconds))/86400
#dfW["days"] = ((dfW['DT'] - bDay).dt.days)
dfW["weeks"] = dfW['days']/7.
printHnT(dfW,3)

In [ ]:
xMin = 0
#yMin = 2.75
yMin = dfWFA["P1"].min()
xPLus = 5
yPlus = 0.1
xMax = dfW['days'].max() + xPLus
yMax = dfW['W'].max() + yPlus

In [ ]:
#Percentile Column Llist
lColsP = [x for x in dfWFA.columns if 'P' in x]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

tAxis = "days"
ax.set_xlim(xMin,xMax)
ax.set_ylim(yMin,yMax)

for i in lColsP:
    ax.plot(dfWFA[tAxis], dfWFA[i], label=str(i), lw=0.5)
labelLines(plt.gca().get_lines(),zorder=2.5)

ax.errorbar(dfW[tAxis], dfW["W"], yerr=dfW["Err"], c="b", label=babyName, lw=1, marker="")
ax.plot(dfW[dfW["officialFlag"]=='y'][tAxis], dfW[dfW["officialFlag"]=='y']["W"], c="r", lw=0, marker="o")
    
ax.set(xlabel='time (d)', ylabel='weight (kg)', title=babyName+'\'s weight')
ax.grid()

dPics["WLarge"]["fName"] = dirData+babyName+"_W.png"
fig.savefig(dPics["WLarge"]["fName"], bbox_inches = 'tight')
plt.show()

In [ ]:
xLookBack = 21 #from xMax

In [ ]:
dfWFA['days'] = dfWFA['days'].astype(float)
#identify main and neighbour percentiles

dSrt = int(dfW["days"].max())+xPLus-xLookBack
dEnd = int(dfW["days"].max())
dfAllDt = pd.concat([dfW[(dSrt <= dfW["days"]) & (dfW["days"] <= dEnd)][['days']],
           dfWFA[(dSrt <= dfWFA["days"]) & (dfWFA["days"] <= dEnd)][['days']]]).sort_values(by='days')
dfAllDt = dfAllDt.merge(dfW[['days', 'W']], on='days', how='left')

distance = 1e6
indexP = 0

#find the closes Percentile
for i,p in enumerate(lColsP):
    dfTMP = dfAllDt.merge(dfWFA[['days', p]], on='days', how='left')
    dfTMP[p] = dfTMP[p].interpolate(method='linear', axis=0).bfill()
    dfTMP['diff'] = abs(dfTMP[p] - dfTMP['W'])
    distTmp = dfTMP['diff'].fillna(0.0).sum()
    if distTmp < distance:
        distance = distTmp
        indexP = i
    if (distTmp > distance) & (i > 0):
        break
        
#Set the 3 percentiles to show:
p1st = ""
p2nd = ""
p3rd = ""

if indexP == 0:
    p1st = lColsP[0]
    p2nd = lColsP[1]
    p3rd = lColsP[2]
elif indexP == (len(lColsP)-1):
    p1st = lColsP[-3]
    p2nd = lColsP[-2]
    p3rd = lColsP[-1]
else:
    p1st = lColsP[indexP-1]
    p2nd = lColsP[indexP]
    p3rd = lColsP[indexP+1]
    
print(p1st, p2nd, p3rd)

In [ ]:
#set plot axis limits
x1 = max(0,xMax-xLookBack)
x2 = xMax
y1 = dfWFA[dfWFA['days'] == math.ceil(x1)][p1st].iloc[0]
y2 = yMax

In [ ]:
tAxis = "days"
try:
    hList = dfW[~dfW['cat'].isnull()]['cat'].astype(int).drop_duplicates().sort_values().unique()
    subFix = " h"
except:
    hList = dfW[~dfW['cat'].isnull()]['cat'].drop_duplicates().sort_values().unique()
    subFix = ""

In [ ]:
#ploat 1st 8 categories
fig, axs = plt.subplots(nrows=4, ncols=2,figsize=(15,15))
axs = np.array(axs)

for i,ax in enumerate(axs.reshape(-1)):
    h=hList[i]
    ax.set_xlim(x1,x2)
    ax.set_ylim(y1,y2)
    ax.plot(dfWFA[tAxis], dfWFA[p1st],  c="b", label=str(p1st), lw=1)
    ax.plot(dfWFA[tAxis], dfWFA[p2nd],  c="g", label=str(p2nd), lw=1)
    ax.plot(dfWFA[tAxis], dfWFA[p3rd],  c="orange", label=str(p3rd), lw=1)
    ax.plot(dfW[dfW["cat"]==h][tAxis], dfW[dfW["cat"]==h]["W"], c="r", label=str(h)+subFix, lw=1, marker="o")
    ax.plot(dfW[tAxis], dfW["W"], c="k", alpha=0.25, lw=1)
    ax.set(ylabel='weight (kg)', title=str(h)+subFix)
    ax.legend(loc='lower right')
    ax.grid()
    
dPics["WZoom"]["fName"] = dirData+babyName+"_W_categ.png"
fig.savefig(dPics["WZoom"]["fName"], bbox_inches = 'tight')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 4))

ax.set_xlim(x1,x2)
ax.set_ylim(y1,y2)
ax.plot(dfWFA[tAxis], dfWFA[p1st],  c="b", label=str(p1st), lw=1)
ax.plot(dfWFA[tAxis], dfWFA[p2nd],  c="g", label=str(p2nd), lw=1)
ax.plot(dfWFA[tAxis], dfWFA[p3rd],  c="orange", label=str(p3rd), lw=1)
ax.plot(dfW[dfW["cat"]==catSel][tAxis], dfW[dfW["cat"]==catSel]["W"], c="r", label=str(h)+subFix, lw=1, marker="o")
ax.plot(dfW[tAxis], dfW["W"], c="k", alpha=0.25, lw=1)
ax.set(xlabel='time (d)', ylabel='weight (kg)', title=str(catSel)+subFix)
ax.legend(loc='lower right')
ax.grid()

dPics["WZoomSelected"]["fName"] = dirData+babyName+"_"+str(catSel)+".png" 
fig.savefig(dPics["WZoomSelected"]["fName"], bbox_inches = 'tight')
plt.show()

---
# Height

In [ ]:
dfHFA.rename(columns={"Day": "days"}, inplace=True)
dfHFA["weeks"] = dfHFA['days']/7.
dfHFA["DT"] = dfHFA["days"].apply(lambda x: babyBDay + dt.timedelta(days=x))
dfHFA["Date"] = dfHFA["DT"].apply(lambda x: x.date())

In [ ]:
dfH = dfH.copy()
dfH["DT"] = pd.to_datetime(dfH["DT"], format='%Y.%m.%d %H:%M')
dfH["Date"] = dfH["DT"].apply(lambda x: x.date())
dfH["days"] = (86400*((dfH['DT'] - babyBDay).dt.days)+((dfH['DT'] - babyBDay).dt.seconds))/86400
dfH["weeks"] = dfH['days']/7.
printHnT(dfH,3)

In [ ]:
xMin = 0
yMin = dfHFA["P1"].min()
xPLus = 5
yPlus = 0.5
xMax = dfH['days'].max() + xPLus
yMax = dfH['H'].max() + yPlus

In [ ]:
#Percentile Column Llist
lColsP = [x for x in dfHFA.columns if 'P' in x]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

tAxis = "days"
ax.set_xlim(xMin,xMax)
ax.set_ylim(yMin,yMax)

for i in lColsP:
    ax.plot(dfHFA[tAxis], dfHFA[i], label=str(i), lw=0.5)
labelLines(plt.gca().get_lines(),zorder=2.5)

ax.plot(dfH[tAxis], dfH["H"], c="b", label=babyName, lw=2, marker=".")
ax.plot(dfH[dfH["officialFlag"]=='y'][tAxis], dfH[dfH["officialFlag"]=='y']["H"], c="r", lw=0, marker="o")
    
ax.set(xlabel='time (days)', ylabel='height (cm)', title=babyName+'\'s height')
ax.grid()

dPics["HLarge"]["fName"] = dirData+babyName+"_H.png"
fig.savefig(dPics["HLarge"]["fName"], bbox_inches = 'tight')
plt.show()

---
# Height to Weight

In [ ]:
#use daily average weight values
dfW["Date"] = dfW["DT"].apply(lambda x: x.date())
dfWDailyAvg = pd.DataFrame(dfW.groupby("Date")["W"].mean()).reset_index()

dfWH = dfWFA[['Date']].drop_duplicates()
dfWH = dfWH.merge(dfWDailyAvg, on=['Date'], how = "left")
dfWH = dfWH.merge(dfH[['Date', 'H']], on=['Date'], how='left')
dfWH = dfWH[~((dfWH["W"].isnull()) | (dfWH["H"].isnull()))]
dfWH["DT"] = pd.to_datetime(dfWH["Date"], format='%Y-%m-%d')
dfWH["years"] = dfWH["DT"].apply(lambda x: relativedelta(x, babyBDay).years)
printHnT(dfWH,3)

## Under 2 years

In [ ]:
dfWH02 = dfWH[dfWH["years"] <= 2]
#dfWFH02

In [ ]:
xMin = dfWFH02["Length"].min()
yMin = dfWFH02["P1"].min()
xPLus = 2
yPlus = 0.5
xMax = dfWH02['H'].max() + xPLus
yMax = dfWH02['W'].max() + yPlus

In [ ]:
#Percentile Column Llist
lColsP = [x for x in dfWFH02.columns if 'P' in x]

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))

tAxis = "Length"
ax.set_xlim(xMin,xMax)
ax.set_ylim(yMin,yMax)

for i in lColsP:
    ax.plot(dfWFH02['Length'], dfWFH02[i], label=str(i), lw=0.5)
labelLines(plt.gca().get_lines(),zorder=2.5)

ax.plot(dfWH02["H"], dfWH02["W"], c="b", label=babyName, lw=2, marker=".")
    
ax.set(xlabel=tAxis, ylabel='weight (kg)', title=babyName+'\'s Length-Weight chart (between 0 and 2 years)')
ax.grid()

dPics["WHLarge"]["fName"] = dirData+babyName+"_LW_02.png"
fig.savefig(dPics["WHLarge"]["fName"], bbox_inches = 'tight')
plt.show()

---
## Above 2 years

In [ ]:
dfWH25 = dfWH[dfWH["years"] >= 2]
#dfWFH25

In [ ]:
if len(dfWH25) > 0:
    xMin = dfWFH25["Length"].min()
    yMin = dfWFH25["P1"].min()
    xPLus = 2
    yPlus = 0.5
    xMax = dfWH25['H'].max() + xPLus
    yMax = dfWH25['W'].max() + yPlus

In [ ]:
#Percentile Column Llist
lColsP = [x for x in dfWFH25.columns if 'P' in x]

In [ ]:
if len(dfWH25) > 0:
    fig, ax = plt.subplots(figsize=(20, 10))

    tAxis = "Height"
    ax.set_xlim(xMin,xMax)
    ax.set_ylim(yMin,yMax)

    for i in lColsP:
        ax.plot(dfWFH02['Height'], dfWFH25[i], label=str(i), lw=0.5)
    labelLines(plt.gca().get_lines(),zorder=2.5)

    ax.plot(dfWH25["H"], dfWH25["W"], c="b", label=babyName, lw=2, marker=".")

    ax.set(xlabel=tAxis, ylabel='weight (kg)', title=babyName+'\'s Height-Weight chart (between 2 and 5 years)')
    ax.grid()

    #sending only this if available (and needed)
    dPics["WHLarge"]["fName"] = dirData+babyName+"_HW_25.png"
    fig.savefig(dPics["WHLarge"]["fName"], bbox_inches = 'tight')
    plt.show()

---
---
# Export CSV: 3 closest percentiles with differences

In [ ]:
dfxport = dfWFA[['days','Date',p1st,p2nd,p3rd]].drop_duplicates()
dfxport['d'] = dfxport['Date'].apply(lambda x: x.strftime('%a'))
dfxport = dfxport.merge(dfW[dfW['cat']==catSel][['Date', 'W']], on=['Date'], how = "left")
dfxport = dfxport[~dfxport['W'].isnull()][['Date','d','days','W', p1st, p2nd,p3rd]]
dfxport['d'+p1st] = dfxport[p1st] - dfxport['W']
dfxport['d'+p2nd] = dfxport[p2nd] - dfxport['W']
dfxport['d'+p3rd] = dfxport[p3rd] - dfxport['W']

dfxport['d'+p1st] = dfxport['d'+p1st].apply(lambda x: round(x, 3))
dfxport['d'+p2nd] = dfxport['d'+p2nd].apply(lambda x: round(x, 3))
dfxport['d'+p3rd] = dfxport['d'+p3rd].apply(lambda x: round(x, 3))
dfxport[p1st] = dfxport[p1st].apply(lambda x: round(x, 3))
dfxport[p2nd] = dfxport[p2nd].apply(lambda x: round(x, 3))
dfxport[p3rd] = dfxport[p3rd].apply(lambda x: round(x, 3))
dfxport['W'] = dfxport['W'].apply(lambda x: round(x, 3))
dfxport.to_csv(dirData+babyName+"_deltas.csv", index=False, sep='\t')
printHnT(dfxport,3)

# Create table to send

In [ ]:
if boolTable:
    dfSend = dfxport.tail(daysInTable).copy()
    dfSendx = dfSend[['Date','d','days','W', 'd'+p1st, 'd'+p2nd,'d'+p3rd]].copy()
    dfSendx.columns = ['Date','d','days','W', p1st, p2nd, p3rd]
    dfSendy = dfSend[['Date','d','days','W', p1st, p2nd, p3rd]].copy()
    dfSendy['Date'] = ''
    dfSendy['d'] = ''
    dfSendy['W'] = ''
    dfSendy['days'] = dfSendy['days']-.5
    dfSend = pd.concat([dfSendx,dfSendy]).sort_values('days', ascending=False)
    dfSend['days'] = dfSend['days'].astype(int)
    dfSend['days'] = np.where(dfSend['d']=='',"", dfSend['days'])
    dfSend.head(4)

## Transform table to HTML code

In [ ]:
if boolTable:
    table = "<table> <tr>"
    for i in dfSend.columns:
        table += "<th>"+i+"</th>"
    table += "</tr><tr>"
    for i,y in enumerate(dfSend['Date']):
        for j,x in enumerate(dfSend.columns):
            table += "<td>"+str(dfSend.iloc[i][j])+"</td>"
        table += "</tr><tr>"
    table += "</tr></table>"
    #print(table)

# sending E-mail

In [ ]:
message = MIMEMultipart("alternative")
message["Subject"] = emailSubject
message["From"] = emailSender
message["To"] = ", ".join(emailReceiver)

# Create the plain-text and HTML version of your message
if not boolTable:
    table = ""

html = """\
<html>
  <body>
    <p>"""+emailBody+"""<br>
    """+table+"""
    </p>
  </body>
</html>
"""

# Turn these into plain/html MIMEText objects
text = MIMEText(html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(text)

for p in list(dPics.keys()):
    if dPics[p]["toSend"]:
        imageFile = workDir+dPics[p]["fName"]
        img_data = open(imageFile, 'rb').read()
        image = MIMEImage(img_data, name=(imageFile))
        message.attach(image)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(emailSender, password)
    server.sendmail(
        emailSender, emailReceiver, message.as_string()
    )
    server.quit()
    
print("> Message sent")